# 2.3

Сопоставление фотографий Прокудина-Горского

Напишите функцию align, которая сопоставляет изображения с фотографий Прокудина-Горского и возвращает координаты точек на синем и красном каналах, как это описано в видео, слайдах и описании задания.
Прототип функции:
def align(img, g_coord):
    row_g, col_g = g_coord
    # считаем сдвиги каналов

    # сдвигаем точку на зеленом канале
    # на другие каналы
    return (row_b, col_b), (row_r, col_r)

Для вашего удобства приводится один тест. Вход: ссылка на изображение, координаты точки на зеленом канале. Выход: правильный ответ — координаты точек на синем и красном каналах.

Программа не должна ничего читать и писать на стандартный ввод и вывод, необходимо написать только функцию align.
Программа тестируется на 10 картинках: 00.png, 01.png, 02.png, 03.png, 04.png, 05.png, 06.png, 07.png, 08.png, 09.png. Однако координаты точек приводятся только для первого изображения. Проверьте перед посылкой решения в систему, что программа верно работает на остальных девяти изображениях. Для этого достаточно проверить, что изображения сопоставляются качественно, без визуальных артефактов.

Sample Input:
https://stepik.org/media/attachments/lesson/58182/00.png 
508,237

Sample Output:
153,236,858,238

https://stepik.org/lesson/58182/step/1?unit=35868

https://stepik.org/lesson/58182/step/2?unit=35868

In [ ]:
def align(img, g_coord):
    row_g, col_g = g_coord

    import numpy as np
    from skimage.io import imread, imsave, imshow
    from skimage import img_as_float, img_as_ubyte
    
    #1. Загружаем файл и преобразовываем
    #img = imread(img)
    
    #2. Преобразуем в вещественные
    img_f = img_as_float(img)
        
    #3.1 Делим на 3 части и записываем в одно RGB изображение
   
    def crop(img):
        height, width = img.shape

        height = img_f.shape[0]//3 #целочисленное деление на 3

        b = img_f[height*0 : height*1, 0 : width]
        g = img_f[height*1 : height*2, 0 : width]
        r = img_f[height*2 : height*3, 0 : width]
        #print(height)
        return np.dstack((r, g, b))
        
    #3.2 Обрезаем края фикс.знач.пикселей
    def cut_fix(image, pix):
        row, cl, n = image.shape
        cut_image = image[pix : row - pix, pix : cl - pix]
        #print(cut_image)
        return cut_image
    
    n = 30 #количество пикселей обрезки
    img_cut = cut_fix(crop(img), n)
    
    h = img_f.shape[0]//3
    h_cut = img_cut.shape[0]
    #print(img_f.shape[0]//3)
    #print(h_cut)
    
    def channel_split(img): #функций нарезки по каналам
        return [img[:, :, i] for i in range(3)]
    
    R, G, B = channel_split(img_cut)
    
    def shift(img_s, img_b):

        shift = {} #словарь, куда складываем координаты смещения и корреляцию

        for rw in range(-15, 16):
            img_shift = np.roll(img_s, rw, axis = 0) # смещение строк

            for cl in range(-15, 16):
                img_shift2 = np.roll(img_shift, cl, axis = 1) # смещение столбцов
                shift[rw, cl] = (img_shift2 * img_b).sum() #считаем корреляцию
        #print(shift) #словарь, куда записываем координаты смещения и корреляцию
        #print(max(shift, key=shift.get))
        r, c = max(shift, key=shift.get)
        return r, c
       
    #возвращаем координаты на исходном изображении
    #по green каналу
    r, c = shift(B, G)
    
    row_b = row_g - r - h #смещаем на высоту необрезанного канала и на расчетное смещение
    col_b = col_g - c #смещаем на расчетное смещение
            
    #по red каналу
    r, c = shift(R, G)
    row_r = row_g - r + h
    col_r = col_g - c

    # сдвигаем точку на зеленом канале
    # на другие каналы
    return (row_b, col_b), (row_r, col_r)